In [9]:
#Import Basic Libraries
import sklearn
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chaimae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chaimae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
#Import Data
# training data
train = pd.read_csv("all.csv", sep=";")
train.head()   

,tweet,label
0,و الله حرام و الله موتوه لشعب الاردني من وين ب...,0
1,البنزين يجي من السعوديه بأقل الاسعار و الحكومه...,0
2,الله يوخذكم,0
3,شو هاد البرنامج معقول عرضوه على القنوات!!!!!!...,0
4,للأسف البعض يعتقد ان المفاعل النووي سيحل مشاكل...,0


In [7]:
#Positive tweets
sum(train["label"] == 1) 

1000

In [8]:
#Negative tweets
sum(train["label"] == 0) 


984

In [6]:
# check if there are any missing values
train.isnull().sum()
train.isnull().values.any()

False

In [11]:
#Importing the necessary libraries
import preprocessor as p
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import ISRIStemmer

# custum function to clean the dataset 

def clean_tweets(data):
 tempArr = []
 for text in data:
    # Suppression des caractères indésirables
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text) #Removing numeric values
    # Expression régulière pour détecter les mots en latin
    latin_pattern = re.compile(r'[a-zA-Z]+')

    # Supprimer les mots en latin
    text = re.sub(latin_pattern, '', text)
    # Word normalization
    stemmer = ISRIStemmer()
    normalized_text = ' '.join(stemmer.stem(word) for word in word_tokenize(text))
    # Remove stop words
    stop_words = set(stopwords.words('arabic'))
    filtered_text = ' '.join(word for word in word_tokenize(normalized_text) if word.lower() not in stop_words)
    tempArr.append( filtered_text)
 return tempArr

In [13]:
# clean training data
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,tweet,label,clean_tweet
0,و الله حرام و الله موتوه لشعب الاردني من وين ب...,0,الل حرم الل وته شعب ارد وين بدن نجب لكو وين ال...
1,البنزين يجي من السعوديه بأقل الاسعار و الحكومه...,0,بنز يجي سعد سعر حكم دبل سعر
2,الله يوخذكم,0,الل وخذ
3,شو هاد البرنامج معقول عرضوه على القنوات!!!!!!...,0,شو هاد رنمج عقل عرضو قنو
4,للأسف البعض يعتقد ان المفاعل النووي سيحل مشاكل...,0,اسف عقد ان فعل نوي يحل شكل اقتصادية ارد ملك ال...
5,المفاعل النووي سيحتاج لشريك استراتيجي يمتلكه ا...,0,فعل نوي حاج لشر استراتيجي ملك او جزء كبر دفع ح...
6,المفاعل النووي سيحتاج لشريك استراتيجي يمتلكه ا...,0,فعل نوي حاج لشر استراتيجي ملك او جزء كبر دفع ح...
7,الطاقة الشمسية هي البديل و ليست النووية,0,طقة شمس بدل نوي
8,للأسف البعض يعتقد ان المفاعل النووي سيحل مشاكل...,0,اسف عقد ان فعل نوي يحل شكل اقتصادية ارد ملك ال...
9,للي مفكرين المفاعل راح يطورهم - المفاعل راح يد...,0,للي فكر فعل يطر فعل دمر ارد نطق الل حول لأن فس...


In [14]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
nltk.download('stopwords')

# Import de la liste de mots vides en arabe
stop_words = stopwords.words('arabic')
vectorizer = CountVectorizer(stop_words=stop_words)


# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

document_term_matrix = vectorizer.fit_transform(x_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chaimae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)
     

In [19]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  81.3758389261745 %
